<a href="https://colab.research.google.com/github/alexnm1/Python/blob/main/Previs%C3%A3o_salarial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Importando Bibliotecas inicializando variáveis

In [1149]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTENC
from sklearn.feature_selection import chi2, SelectKBest
from sklearn.preprocessing import MinMaxScaler
from scipy import stats
from sklearn.ensemble import IsolationForest

In [1150]:
def substituir_valor(dataset, coluna, valor_antigo, valor_novo):
    if coluna in dataset.columns:
        dataset[coluna].replace(valor_antigo, valor_novo, inplace=True)
    else:
        raise ValueError(f"A coluna '{coluna}' não existe no dataset.")

In [1151]:
def detect_and_replace_outliers(df, column_name, threshold):
    z_scores = np.abs(stats.zscore(df[column_name]))
    anomalies = df[np.abs(z_scores) > threshold]
    median_value = df[np.abs(z_scores) <= threshold][column_name].median()
    df[column_name][np.abs(z_scores) > threshold] = median_value
    return df

In [1152]:
def treat_outliers_with_isolation_forest(df, columns, n_estimators=100, contamination=0.05, random_state=42):
    model = IsolationForest(n_estimators=n_estimators, contamination=contamination, random_state=random_state)

    for column in columns:
        # Ajustar o modelo aos dados e prever anomalias
        model.fit(df[[column]])
        predictions = model.predict(df[[column]])

        # Substituir os valores anômalos pela mediana
        median_value = df[column].median()
        df[column][predictions == -1] = median_value

    return df

In [1153]:
def aplicar_label_encoder(df, colunas):
    labelencoder = LabelEncoder()
    for coluna in colunas:
          df[coluna] = labelencoder.fit_transform(df[coluna])
    return df

In [1154]:
dataset = pd.read_csv('train.csv')
datasetValidacao = pd.read_csv('validation.csv')

In [1155]:
dataset.head()

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,25,Private,188767,HS-grad,9,Never-married,Craft-repair,Own-child,White,Male,0,0,40,United-States,<=50K
1,64,Private,286732,HS-grad,9,Widowed,Sales,Not-in-family,White,Female,0,0,17,United-States,<=50K
2,29,Private,253801,HS-grad,9,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,40,Ecuador,<=50K
3,28,Private,334032,Assoc-voc,11,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,50,United-States,<=50K
4,22,Private,173004,HS-grad,9,Never-married,Machine-op-inspct,Other-relative,Black,Male,0,0,1,United-States,<=50K


In [1156]:
y = dataset['income']
x = dataset.iloc[:,:-1]
y_validacao = datasetValidacao['income']
x_validacao = datasetValidacao.iloc[:,:-1]

### Verificando valores nulos ou com ?


In [ ]:
#verificando valores nulos
x.isnull().sum()

age                0
workclass          0
fnlwgt             0
education          0
educational-num    0
marital-status     0
occupation         0
relationship       0
race               0
gender             0
capital-gain       0
capital-loss       0
hours-per-week     0
native-country     0
dtype: int64

In [ ]:
#verificando valores com ?
(x == "?").sum()

age                   0
workclass          1969
fnlwgt                0
education             0
educational-num       0
marital-status        0
occupation         1977
relationship          0
race                  0
gender                0
capital-gain          0
capital-loss          0
hours-per-week        0
native-country      585
dtype: int64

In [ ]:
#verificando valores nulos
x_validacao.isnull().sum()

age                0
workclass          0
fnlwgt             0
education          0
educational-num    0
marital-status     0
occupation         0
relationship       0
race               0
gender             0
capital-gain       0
capital-loss       0
hours-per-week     0
native-country     0
dtype: int64

In [ ]:
#verificando valores com ?
(x_validacao == "?").sum()

age                  0
workclass          404
fnlwgt               0
education            0
educational-num      0
marital-status       0
occupation         405
relationship         0
race                 0
gender               0
capital-gain         0
capital-loss         0
hours-per-week       0
native-country     138
dtype: int64

### Tratando valores com ?, usando Moda

In [1157]:
agrupado = x_validacao.groupby(['workclass']).size()
agrupado

workclass
?                    404
Federal-gov          211
Local-gov            469
Never-worked           1
Private             5084
Self-emp-inc         250
Self-emp-not-inc     591
State-gov            314
Without-pay            2
dtype: int64

In [1158]:
substituir_valor(x, 'workclass', '?', 'Private')
substituir_valor(x_validacao, 'workclass', '?', 'Private')

In [1159]:
agrupado = x.groupby(['workclass']).size()
agrupado

workclass
Federal-gov          1009
Local-gov            2194
Never-worked            8
Private             25688
Self-emp-inc         1197
Self-emp-not-inc     2687
State-gov            1389
Without-pay            17
dtype: int64

In [1160]:
substituir_valor(x, 'native-country', '?', 'United-States')
substituir_valor(x_validacao, 'native-country', '?', 'United-States')

In [1161]:
agrupado = x.groupby(['native-country']).size()
agrupado

native-country
Cambodia                         25
Canada                          131
China                            96
Columbia                         55
Cuba                             86
Dominican-Republic               69
Ecuador                          31
El-Salvador                     110
England                          75
France                           32
Germany                         138
Greece                           39
Guatemala                        63
Haiti                            54
Holand-Netherlands                1
Honduras                         13
Hong                             21
Hungary                          14
India                           105
Iran                             44
Ireland                          25
Italy                            76
Jamaica                          73
Japan                            57
Laos                             18
Mexico                          670
Nicaragua                        39
Outlying-US(G

In [1162]:
agrupado = x.groupby(['occupation']).size()
agrupado

occupation
?                    1977
Adm-clerical         3918
Armed-Forces           11
Craft-repair         4282
Exec-managerial      4256
Farming-fishing      1030
Handlers-cleaners    1418
Machine-op-inspct    2137
Other-service        3459
Priv-house-serv       180
Prof-specialty       4333
Protective-serv       714
Sales                3880
Tech-support          954
Transport-moving     1640
dtype: int64

### Label Encoder


In [1163]:
colunas_para_codificar = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'gender', 'native-country']
x = aplicar_label_encoder(x, colunas_para_codificar)
x_validacao = aplicar_label_encoder(x_validacao, colunas_para_codificar)
y = labelencoder.fit_transform(y)
y_validacao = labelencoder.fit_transform(y_validacao)

x_validacao

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country
0,22,3,174043,11,9,4,3,1,4,1,0,0,50,37
1,50,6,159219,9,13,2,4,0,4,1,0,0,40,1
2,18,3,192399,15,10,4,0,3,4,1,0,0,60,37
3,33,0,193246,9,13,2,1,0,4,1,0,0,42,37
4,23,3,61777,9,13,2,12,0,4,1,2580,0,40,37
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7321,47,0,211123,11,9,2,3,0,4,1,0,0,40,37
7322,51,1,123011,12,14,2,10,0,4,1,0,1977,35,37
7323,21,3,195919,0,6,4,6,1,3,1,0,0,40,5
7324,56,0,255386,15,10,2,1,0,1,1,0,0,40,23


### Balanceamento de classe

In [1164]:
from imblearn.over_sampling import SMOTE

smote = SMOTE()
x, y = smote.fit_resample(x, y)

smote = SMOTE()
x_validacao, y_validacao = smote.fit_resample(x_validacao, y_validacao)

### Outliers usando Z-Score pra tratar anomalias

In [1165]:
x = detect_and_replace_outliers(x, 'hours-per-week', 5.0)
x_validacao = detect_and_replace_outliers(x_validacao, 'hours-per-week', 5.0)

In [1166]:
x = detect_and_replace_outliers(x, 'educational-num', 5.0)
x_validacao = detect_and_replace_outliers(x_validacao, 'educational-num', 5.0)

In [1167]:
x = detect_and_replace_outliers(x, 'fnlwgt', 5.0)
x_validacao = detect_and_replace_outliers(x_validacao, 'fnlwgt', 5.0)

In [1168]:
x = detect_and_replace_outliers(x, 'age', 5.0)
x_validacao = detect_and_replace_outliers(x_validacao, 'age', 5.0)

### Outliers usando IsolationForest pra tratar anomalias(Não utilizada)

In [ ]:
x = treat_outliers_with_isolation_forest(x, ['age', 'hours-per-week','fnlwgt','educational-num'])
x_validacao = treat_outliers_with_isolation_forest(x_validacao, ['age', 'hours-per-week','fnlwgt','educational-num'])

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names,

### Dimensionamento da escala dos Atributos

In [1169]:
scaler = MinMaxScaler(feature_range=(0,1))
x_escalavel_treino = scaler.fit_transform(x)

scaler = MinMaxScaler(feature_range=(0,1))
x_escalavel_validacao = scaler.fit_transform(x_validacao)
x_escalavel_validacao

array([[0.06849315, 0.42857143, 0.23125666, ..., 0.        , 0.5       ,
        0.94871795],
       [0.45205479, 0.85714286, 0.20910255, ..., 0.        , 0.39795918,
        0.02564103],
       [0.01369863, 0.42857143, 0.25868926, ..., 0.        , 0.60204082,
        0.94871795],
       ...,
       [0.28767123, 0.42857143, 0.35234057, ..., 0.        , 0.35714286,
        0.46153846],
       [0.30136986, 0.42857143, 0.35420867, ..., 0.        , 0.44897959,
        0.94871795],
       [0.31506849, 0.        , 0.73278924, ..., 0.        , 0.39795918,
        0.94871795]])

### Selecao de Atributos

In [ ]:
selecao = SelectKBest(chi2, k=7)
x_novo_validacao =  selecao.fit_transform(x_escalavel_validacao, y_validacao)
x_novo_treino =  selecao.fit_transform(x_escalavel_treino, y)

In [ ]:
x_novo_validacao

array([[0.06849315, 0.53333333, 0.66666667, ..., 1.        , 0.        ,
        0.        ],
       [0.45205479, 0.8       , 0.33333333, ..., 1.        , 0.        ,
        0.        ],
       [0.01369863, 0.6       , 0.66666667, ..., 1.        , 0.        ,
        0.        ],
       ...,
       [0.30136986, 0.53333333, 0.33333333, ..., 1.        , 0.        ,
        0.        ],
       [0.46575342, 0.53333333, 0.33333333, ..., 1.        , 0.        ,
        0.        ],
       [0.35616438, 0.8       , 0.33333333, ..., 0.        , 0.        ,
        0.06037649]])

In [ ]:
#Treina
floresta = RandomForestClassifier(n_estimators=100, random_state = 1234)
floresta.fit(x_novo_treino,y)

#Valida com dados de validacao com selecao de atributos
previsoes = floresta.predict(x_novo_validacao)
taxa_acerto = f1_score(y_validacao, previsoes, average='weighted')
taxa_acerto

0.8485457102639999

### Treino do Modelo

In [1170]:
# Treina o modelo
floresta = RandomForestClassifier(n_estimators=100, random_state=1234)
floresta.fit(x_escalavel_treino, y)

# Valida com dados de validação
previsaoRandomForestClassifier = floresta.predict(x_escalavel_validacao)

#Mostra 10 primeiras linhas
print("10 primeiras linhas:", y_validacao[0:10])

#Mostra predicoes
print("Predições:", previsaoRandomForestClassifier[0:10])

# Calcula o F1-Score ponderado
f1_score_result = f1_score(y_validacao, previsaoRandomForestClassifier, average='weighted')

# Calcula a Accuracy
accuracy_result = accuracy_score(y_validacao, previsaoRandomForestClassifier)

# Imprime os resultados
print("F1-Score:", f1_score_result)
print("Accuracy (Precisão):", accuracy_result)

10 primeiras linhas: [0 1 0 1 0 0 0 0 0 0]
Predições: [0 1 0 1 0 0 0 1 0 0]
F1-Score: 0.8809532882719923
Accuracy (Precisão): 0.8809819213045019


In [1171]:
report = classification_report(y_validacao, previsaoRandomForestClassifier)
print(report)

              precision    recall  f1-score   support

           0       0.87      0.90      0.88      5642
           1       0.89      0.87      0.88      5642

    accuracy                           0.88     11284
   macro avg       0.88      0.88      0.88     11284
weighted avg       0.88      0.88      0.88     11284



### Tuning(Usando TPOTClassifier)

In [1172]:
!pip install tpot
from tpot import TPOTClassifier

In [1173]:
from traitlets.traitlets import ForwardDeclaredInstance
tpot = TPOTClassifier(
    generations = 20,
    population_size = 200,
    offspring_size = 200,
    mutation_rate = 0.8,
    crossover_rate = 0.2,
    scoring='f1',
    max_time_mins = 5,
    random_state = 0,
    early_stop= False,
    verbosity = 2
)

tpot.fit(x_escalavel_treino, y)
print("Melhor modelo: ", tpot.fitted_pipeline_)

Optimization Progress:   0%|          | 0/200 [00:00<?, ?pipeline/s]


8.38 minutes have elapsed. TPOT will close down.
TPOT closed during evaluation in one generation.


TPOT closed prematurely. Will use the current best pipeline.

Best pipeline: RandomForestClassifier(input_matrix, bootstrap=False, criterion=gini, max_features=0.4, min_samples_leaf=2, min_samples_split=14, n_estimators=100)
Melhor modelo:  Pipeline(steps=[('randomforestclassifier',
                 RandomForestClassifier(bootstrap=False, max_features=0.4,
                                        min_samples_leaf=2,
                                        min_samples_split=14,
                                        random_state=0))])


In [1175]:
previsaoTPOT = tpot.predict(x_escalavel_validacao)

# Calcula o F1-Score ponderado
f1_score_result = f1_score(y_validacao, previsaoTPOT, average='weighted')

# Calcula a Accuracy
accuracy_result = accuracy_score(y_validacao, previsaoTPOT)

# Imprime os resultados
print("F1-Score:", f1_score_result)
print("Accuracy (Precisão):", accuracy_result)

F1-Score: 0.884506626156545
Accuracy (Precisão): 0.8845267635590216


### Teste Tuning H2O

In [ ]:
!pip install h2o
import h2o
from h2o.automl import H2OAutoML

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.9/240.9 MB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for h2o: filename=h2o-3.42.0.3-py2.py3-none-any.whl size=240945068 sha256=549233a84614a2c5b11d6a11d30db506557a18dd1429e41ed188b4f393dabfe8
  Stored in directory: /root/.cache/pip/wheels/f9/7d/1d/e1258f0558603969a8393cb5b22a18e313337f01627a118302
Successfully built h2o


In [ ]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.20.1" 2023-08-24; OpenJDK Runtime Environment (build 11.0.20.1+1-post-Ubuntu-0ubuntu122.04); OpenJDK 64-Bit Server VM (build 11.0.20.1+1-post-Ubuntu-0ubuntu122.04, mixed mode, sharing)
  Starting server from /usr/local/lib/python3.10/dist-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpn2nwbcy5
  JVM stdout: /tmp/tmpn2nwbcy5/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmpn2nwbcy5/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,10 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.42.0.3
H2O_cluster_version_age:,1 month and 10 days
H2O_cluster_name:,H2O_from_python_unknownUser_oq2gwv
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.170 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"locked, healthy"


In [ ]:
datasetTreino = h2o.import_file('train.csv')
datasetValidacao = h2o.import_file('validation.csv')

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [ ]:
model_autom1 = H2OAutoML(max_runtime_secs=20,sort_metric = "f1")
model_autom1.train(y="income", training_frame=datasetTreino)

AutoML progress: |███████████████████████████████████████████████████████████████| (done) 100%


Model Details
=============
H2OXGBoostEstimator : XGBoost
Model Key: XGBoost_2_AutoML_1_20231002_162734


Model Summary: 
    number_of_trees
--  -----------------
    1

ModelMetricsBinomial: xgboost
** Reported on train data. **

MSE: 0.17478919721153757
RMSE: 0.41807797982139355
LogLoss: 0.5393519680320892
Mean Per-Class Error: 0.18728053408651918
AUC: 0.9041953043370504
AUCPR: 0.7881497181643407
Gini: 0.8083906086741008

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.46224266290664673
       <=50K    >50K    Error    Rate
-----  -------  ------  -------  ----------------
<=50K  15388    2751    0.1517   (2751.0/18139.0)
>50K   1281     4466    0.2229   (1281.0/5747.0)
Total  16669    7217    0.1688   (4032.0/23886.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value     idx
---------------------------  -----------  --------  -----
max f1                       0.462243     0.688985  43
max f2                       0.40437      0.786901  64
max f0point5                 0.529964     0.733112  22
max accuracy                 0.493047     0.857783  33
max precision                0.643517     1         0
max recall                   0.357046     1         77
max specificity              0.643517     1         0
max absolute_mcc             0.491667     0.588234  34
max min_per_class_accuracy   0.442562     0.816774  52
max mean_per_class_accuracy  0.420676     0.819527  60
max tns                      0.643517     18139     0
max fns                      0.643517     5315      0
max fps                      0.357046     18139     77
max tps                      0.357046     5747      77
max tnr                      0.643517     1         0
max fnr                      0.643517     0.92483   0
max fpr                      0.357046     1         77
max tpr                      0.357046     1         77

Gains/Lift Table: Avg response rate: 24.06 %, avg score: 42.47 %
group    cumulative_data_fraction    lower_threshold    lift       cumulative_lift    response_rate    score     cumulative_response_rate    cumulative_score    capture_rate    cumulative_capture_rate    gain      cumulative_gain    kolmogorov_smirnov
-------  --------------------------  -----------------  ---------  -----------------  ---------------  --------  --------------------------  ------------------  --------------  -------------------------  --------  -----------------  --------------------
1        0.0180859                   0.643517           4.15626    4.15626            1                0.643517  1                           0.643517            0.0751697       0.0751697                  315.626   315.626            0.0751697
2        0.0296408                   0.639013           4.12614    4.14451            0.992754         0.639013  0.997175                    0.641761            0.047677        0.122847                   312.614   314.451            0.122736
3        0.0430796                   0.638549           4.07857    4.12394            0.981308         0.638549  0.992225                    0.640759            0.0548112       0.177658                   307.857   312.394            0.177217
4        0.0547601                   0.634471           4.05198    4.10859            0.97491          0.634471  0.988532                    0.639418            0.047329        0.224987                   305.198   310.859            0.22416
5        0.106422                    0.577976           3.36475    3.7475             0.809562         0.592999  0.901652                    0.616884            0.17383         0.398817                   236.475   274.75             0.385034
6        0.168425                    0.514132           2.47523    3.27913            0.595544         0.529125  0.788963                    0.584577            0.153471        0.552288                   147.523   227.913            0.505483
7        0.200368                    0.483339           2.04817    3.08289

In [ ]:
ranking = model_autom1.leaderboard
ranking = ranking.as_data_frame()
ranking

,model_id,f1,auc,logloss,aucpr,mean_per_class_error,rmse,mse
0,XGBoost_2_AutoML_1_20231002_162734,0.699229,0.901174,0.540125,0.783954,0.181338,0.418541,0.175177
1,GLM_1_AutoML_1_20231002_162734,0.704927,0.908085,0.320634,0.773913,0.185055,0.320547,0.102751
2,GBM_1_AutoML_1_20231002_162734,0.730667,0.920831,0.300005,0.810850,0.164070,0.307565,0.094596
3,StackedEnsemble_BestOfFamily_1_AutoML_1_202310...,0.735880,0.924166,0.291768,0.818058,0.161783,0.303751,0.092265
4,XGBoost_1_AutoML_1_20231002_162734,0.736898,0.923315,0.292871,0.817408,0.160014,0.304154,0.092510


### Redes Neurais

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
import keras.backend as K

In [ ]:
def f1_metric(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

In [ ]:
modelo = Sequential()
modelo.add(Dense(units=128, activation='relu', input_dim=x_escalavel_treino.shape[1]))
modelo.add(Dropout(0.5))
modelo.add(Dense(units=64, activation='relu'))
modelo.add(Dropout(0.4))
modelo.add(Dense(units=32, activation='relu'))
modelo.add(Dropout(0.4))
modelo.add(Dense(units=1, activation='sigmoid'))

modelo.compile(optimizer='adam', loss='binary_crossentropy', metrics=[f1_metric])
modelo.fit(x_escalavel_treino, y, epochs=150, batch_size=128)

Epoch 1/150
406/406 [==============================] - 3s 4ms/step - loss: 0.5403 - f1_metric: 0.7311
Epoch 2/150
406/406 [==============================] - 1s 3ms/step - loss: 0.4205 - f1_metric: 0.8211
Epoch 3/150
406/406 [==============================] - 1s 4ms/step - loss: 0.3978 - f1_metric: 0.8274
Epoch 4/150
406/406 [==============================] - 2s 5ms/step - loss: 0.3908 - f1_metric: 0.8315
Epoch 5/150
406/406 [==============================] - 2s 5ms/step - loss: 0.3819 - f1_metric: 0.8352
Epoch 6/150
406/406 [==============================] - 1s 4ms/step - loss: 0.3757 - f1_metric: 0.8383
Epoch 7/150
406/406 [==============================] - 1s 4ms/step - loss: 0.3742 - f1_metric: 0.8393
Epoch 8/150
406/406 [==============================] - 1s 3ms/step - loss: 0.3717 - f1_metric: 0.8403
Epoch 9/150
406/406 [==============================] - 2s 4ms/step - loss: 0.3683 - f1_metric: 0.8408
Epoch 10/150
406/406 [==============================] - 2s 5ms/step - loss: 0.3652

In [ ]:
previsaoRedeNeural = modelo.predict(x_escalavel_validacao)
previsaoRedeNeural

353/353 [==============================] - 1s 2ms/step


array([[0.00712498],
       [0.47789547],
       [0.00194431],
       ...,
       [0.90563905],
       [0.77768207],
       [0.89779735]], dtype=float32)

In [ ]:
y_pred= (previsaoRedeNeural > 0.5).astype('int32')
y_pred

array([[0],
       [0],
       [0],
       ...,
       [1],
       [1],
       [1]], dtype=int32)

In [ ]:
# Calcula o F1-Score ponderado
f1_score_result = f1_score(y_validacao, y_pred, average='weighted')

# Calcula a Accuracy
accuracy_result = accuracy_score(y_validacao, y_pred)

# Imprime os resultados
print("F1-Score:", f1_score_result)
print("Accuracy (Precisão):", accuracy_result)

F1-Score: 0.8621756817795596
Accuracy (Precisão): 0.8621942573555477


### Teste modelo

In [1177]:
nova_linha = {
    'age': 37,
    'workclass': 3,
    'fnlwgt': 174043.0,
    'education': 11,
    'educational-num': 9,
    'marital-status': 2,
    'occupation': 3,
    'relationship': 1,
    'race': 4,
    'gender': 1,
    'capital-gain': 0,
    'capital-loss': 0,
    'hours-per-week': 40,
    'native-country': 1
}


nova_linha_df = pd.DataFrame([nova_linha])
x_validacao = x_validacao.append(nova_linha_df, ignore_index=True)
scaler = MinMaxScaler(feature_range=(0,1))
x_escalavel_validacao = scaler.fit_transform(x_validacao)

# Valida com dados de validação
previsaoRandomForestClassifier = floresta.predict(x_escalavel_validacao)

#Mostra predicao
ultima_predicao = previsaoRandomForestClassifier[-1]

if ultima_predicao == 0:
    print("Predição: Ganha menos de 50 mil.")
elif ultima_predicao == 1:
    print("Predição: Ganha mais de 50 mil.")

<ipython-input-1177-1a9e10b87c6b>:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  x_validacao = x_validacao.append(nova_linha_df, ignore_index=True)


Predição: Ganha menos de 50 mil.


### Histórico de Tunning

  ### Sem nenhum pré-processamento dos dados, sem tratamento de desbalançeamento, sem tratamento de anomalias
    RandomForest: F1Score = 0.8573590583072475, accuracy = 0.8610428610428611
    AutoML: F1Score = 0.8637021999132661, accuracy = 0.8674583674583675
    RedeNeurais : F1Score = 0.0.8500227934524871, accuracy = 0.8595413595413596

  ### Somente tratando valores nulos ou com ?
    RandomForest: F1Score = 0.8597182032123398, accuracy = 0.862953862953863
    AutoML: F1Score = 0.8669106812840867 , accuracy = 0.871007371007371
    RedeNeurais : F1Score = 0.8455046848909247 , accuracy = 0.8562653562653563

### Somente tratando valores nulos ou com ? e tratando anomalias
    RandomForest: F1Score = 0.8594405813090137, accuracy = 0.862953862953863
    AutoML: F1Score = 0.864289887538663, accuracy = 0.8677313677313677
    RedeNeurais : F1Score = 0.8491050336251826, accuracy = 0.8569478569478569


### Somente tratando valores nulos ou com ? e Dimensionamento da escala dos Atributos , com balanceamento de classes usando SMOTE  
    RandomForest: F1Score = 0.8846023339287314, accuracy = 0.8846153846153846
    RandomForest(AutoML): F1Score = 0.8850400273094727,accuracy = 0.8850584898971996
    RedeNeurais : F1Score = 0.8621756817795596, accuracy = 0.8621942573555477

### Eli5(Explicando modelo escolhido)

In [ ]:
!pip install eli5

import eli5
from eli5 import show_prediction

In [ ]:
eli5.show_weights(floresta, feature_names = list(dataset)[0:14])

Weight,Feature
0.1639 ± 0.1915,marital-status
0.1359 ± 0.1074,age
0.1194 ± 0.1657,relationship
0.1103 ± 0.0058,fnlwgt
0.0978 ± 0.0830,hours-per-week
0.0855 ± 0.0547,educational-num
0.0787 ± 0.0393,education
0.0651 ± 0.0305,capital-gain
0.0569 ± 0.0196,occupation
0.0335 ± 0.0103,workclass


### Conclusões

  O Modelo treinado pra obter a previsão salarial em cima dos dados da planilha "train.csv", usando RandomForest com pré-processamento de dados obteve um f1-score e accuracy otimizado de 88%, outros algoritmos foram testados com diversos hiper-parametros, mas esse se mostrou o melhor modelo.

  Redes neurais pra essa massa de dados obteve piores resultados..

  As caracteristicas que mais impactam na predição do modelo são "marital-status" e "age"